# 💬 ChipChat - 데이터시트 챗봇

**전처리된 데이터시트 JSON 파일을 기반으로 질의응답을 수행하는 챗봇입니다.**

---

## 🚀 시작하기 전에

이 노트북은 Google Colab에서 Streamlit 애플리케이션을 실행하는 방법을 제공합니다.
PDF 전처리 모듈에서 생성된 JSON 파일을 사용합니다.

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 전처리된 JSON 파일과 벡터 스토어를 관리합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# JSON 및 벡터 스토어 폴더 설정
json_folder = Path('/content/drive/MyDrive/processed_json')
vectorstore_folder = Path('/content/drive/MyDrive/vectorstore')
prompt_templates_folder = Path('/content/drive/MyDrive/prompt_templates')

# 폴더가 없으면 생성
json_folder.mkdir(parents=True, exist_ok=True)
vectorstore_folder.mkdir(parents=True, exist_ok=True)
prompt_templates_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 JSON 폴더: {json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")
print(f"📁 프롬프트 템플릿 폴더: {prompt_templates_folder}")

# JSON 파일 확인
json_files = list(json_folder.glob("*.json"))
print(f"\n📄 발견된 JSON 파일: {len(json_files)}개")
for json_file in json_files[:5]:  # 처음 5개만 표시
    print(f" - {json_file.name}")
    
if len(json_files) > 5:
    print(f" ... 외 {len(json_files) - 5}개")

if not json_files:
    print("\n⚠️ JSON 파일이 없습니다. PDF 전처리 모듈을 먼저 실행하여 JSON 파일을 생성해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

!pip install -r requirements.txt -q
!pip install pyngrok==7.0.1 -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

print("✅ 라이브러리 설치 완료!")

In [ ]:
#@title 🔑 3단계: OpenAI API 키 설정 { display-mode: "form" }
#@markdown OpenAI API 키를 입력하여 챗봇을 설정합니다.

import os
import json

api_key = "" #@param {type:"string"}

if not api_key:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 OpenAI API 키를 입력해주세요. (https://platform.openai.com/api-keys)")
else:
    os.environ["OPENAI_API_KEY"] = api_key
    
    # Streamlit 시크릿 파일 생성
    secrets_dir = Path(".streamlit")
    secrets_dir.mkdir(exist_ok=True)
    
    secrets = {
        "openai_api_key": api_key
    }
    
    with open(secrets_dir / "secrets.toml", "w") as f:
        f.write(f'openai_api_key = "{api_key}"\n')
    
    print("✅ API 키 설정 완료!")
    
    # 기본 프롬프트 템플릿 생성
    default_template = {
        "pre": "당신은 전자 부품 데이터시트에 대해 응답하는 도우미입니다. 제공된 컨텍스트 정보를 기반으로 질문에 답변하세요.",
        "post": "검색된 정보를 바탕으로 명확하고 간결하게 답변해주세요."
    }
    
    template_file = prompt_templates_folder / "default_template.json"
    if not template_file.exists():
        with open(template_file, 'w', encoding='utf-8') as f:
            json.dump(default_template, f, ensure_ascii=False, indent=2)
            
        print("✅ 기본 프롬프트 템플릿이 생성되었습니다.")

In [ ]:
#@title 📊 4단계: Streamlit 서버 실행 { display-mode: "form" }
#@markdown Streamlit 서비스를 실행하여 웹 인터페이스를 제공합니다.

import subprocess
import threading
import time
import os
import json
from pathlib import Path
from pyngrok import ngrok

# 환경 변수 설정
os.environ['VECTORSTORE_PATH'] = str(vectorstore_folder)
os.environ['JSON_FOLDER_PATH'] = str(json_folder)
os.environ['PROMPT_TEMPLATES_PATH'] = str(prompt_templates_folder)

# Streamlit 실행 함수
def run_streamlit():
    cmd = [
        "streamlit", "run", 
        "main.py",
        "--server.port=8501", 
        "--server.address=localhost"
    ]
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    return process

# Ngrok 설정 (공개 URL 만들기)
ngrok_token = "" #@param {type:"string"}

if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

# Streamlit 서버 실행
print("🚀 Streamlit 서버를 시작합니다...")
process = run_streamlit()
time.sleep(5)  # 서버 시작 대기

# Ngrok 터널 설정
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n✅ 서버가 시작되었습니다!")
    print(f"\n🔗 다음 URL을 통해 ChipChat에 접속할 수 있습니다:")
    print(f"📱 {public_url}")
    print("\n이 URL은 세션이 유지되는 동안에만 유효합니다.")
    print("URL을 클릭하면 ChipChat 웹 인터페이스로 이동합니다.")
except Exception as e:
    # ngrok이 실패하면 Colab의 웹 프록시 사용
    print("\n⚠️ ngrok 연결 중 오류가 발생했습니다. Colab의 웹 프록시를 사용합니다.")
    
    from google.colab.output import eval_js
    url = eval_js('google.colab.kernel.proxyPort(8501)')
    
    print(f"\n✅ 서버가 시작되었습니다!")
    print(f"\n🔗 다음 URL을 통해 ChipChat에 접속할 수 있습니다:")
    print(f"📱 {url}")

In [ ]:
#@title 🛑 5단계: 서버 중지 { display-mode: "form" }
#@markdown 작업을 마치면 서버를 중지합니다.

import os
import signal
from pyngrok import ngrok

# 서버 중지 여부 확인
stop_server = True  #@param {type:"boolean"}

if stop_server:
    # ngrok 터널 종료
    try:
        ngrok.kill()
        print("✅ ngrok 터널이 종료되었습니다.")
    except:
        pass
    
    # Streamlit 프로세스 종료
    try:
        # 모든 streamlit 프로세스 종료
        !pkill -f streamlit
        print("✅ Streamlit 서버가 종료되었습니다.")
    except:
        print("⚠️ 서버 종료 중 오류가 발생했습니다. 런타임을 재시작하는 것을 권장합니다.")
else:
    print("ℹ️ 서버가 계속 실행 중입니다.")

## 📝 ChipChat 사용 가이드

ChipChat은 다음 세 가지 모드로 동작합니다:

1. **Vectorstore Creation**: 
   - 전처리된 JSON 파일을 바탕으로 벡터 스토어를 생성합니다.
   - 이 작업은 처음 한 번만 수행하면 됩니다.

2. **Retrieval Test**: 
   - 검색 파라미터(k, threshold)를 설정하고 테스트합니다.
   - 설정에 따른 검색 결과를 확인할 수 있습니다.

3. **Chat**: 
   - 실제 챗봇과 대화할 수 있는 인터페이스입니다.
   - 프롬프트 템플릿을 선택하거나 수정할 수 있습니다.
   
모든 설정과 데이터는 Google Drive에 저장되므로 다음 세션에서도 이어서 사용할 수 있습니다.